In [37]:
from transformers import BartTokenizer, BartForConditionalGeneration, BartModel
from transformers.modeling_outputs import BaseModelOutput
from IPython.display import display, HTML
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [38]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [39]:
model_name = "facebook/bart-large"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name).to(device)

loading file vocab.json from cache at C:\Users\sreya\.cache\huggingface\hub\models--facebook--bart-large\snapshots\cb48c1365bd826bd521f650dc2e0940aee54720c\vocab.json
loading file merges.txt from cache at C:\Users\sreya\.cache\huggingface\hub\models--facebook--bart-large\snapshots\cb48c1365bd826bd521f650dc2e0940aee54720c\merges.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at C:\Users\sreya\.cache\huggingface\hub\models--facebook--bart-large\snapshots\cb48c1365bd826bd521f650dc2e0940aee54720c\tokenizer_config.json
loading file tokenizer.json from cache at C:\Users\sreya\.cache\huggingface\hub\models--facebook--bart-large\snapshots\cb48c1365bd826bd521f650dc2e0940aee54720c\tokenizer.json
loading configuration file config.json from cache at C:\Users\sreya\.cache\huggingface\hub\models--facebook--bart-large\snapshots\cb48c1365bd826bd521f650dc2e0940aee54720c\config.json
Model config 

In [40]:
original_text = """
Welcome to our online coding bootcamp program! 
Whether you're a complete beginner or looking to improve your programming skills, this course is designed for you. 
Throughout the course, you will learn essential coding languages such as Python and JavaScript. 
Our instructors will guide you through interactive projects and provide real-time feedback. 
Each student will receive a certificate of completion at the end of the program. 
The total cost for the bootcamp is $150, which includes all learning materials.
"""

input_text = """
Welcome to our online <mask> bootcamp program! 
Whether you're a complete beginner or looking to <mask> your programming skills, this course is designed for you. 
Throughout the course, you will learn essential <mask> such as Python and JavaScript. 
Our instructors will guide you through interactive projects and provide real-time <mask>. 
Each student will receive a certificate of completion at the end of the <mask>. 
The total cost for the bootcamp is $150, which <mask> all learning materials.
""".replace("\n", "")

In [41]:
# return_tensors returns the tokenizer output as pytorch tensors
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)
print(input_ids.shape)

torch.Size([1, 94])


In [42]:
with torch.no_grad():
    encoder_outputs = model.model.encoder(input_ids=input_ids)

print(encoder_outputs)

BaseModelOutput(last_hidden_state=tensor([[[-0.0008,  0.0308,  0.0059,  ...,  0.0247, -0.0319,  0.0094],
         [-0.0300, -0.0781,  0.0877,  ..., -0.2132,  0.0201,  0.0499],
         [-0.0061,  0.1724,  0.1160,  ..., -0.1056, -0.0867, -0.0710],
         ...,
         [-0.0306, -0.1893, -0.0245,  ..., -0.1376, -0.0109, -0.0656],
         [ 0.0008,  0.0165,  0.0067,  ...,  0.0070, -0.0087, -0.0018],
         [ 0.1437,  0.2210,  0.0996,  ..., -0.0200, -0.0611,  0.0710]]],
       device='cuda:0'), hidden_states=None, attentions=None)


In [43]:
no_noise_outputs = model.generate(
    input_ids = None,
    encoder_outputs = encoder_outputs,
    max_length = 200,
    min_length = 10,
    do_sample = True,
    temperature = 0.1
)

print(no_noise_outputs[0])

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1
}



tensor([    2,     0, 25194,     7,    84,   804,  8326,  9759, 21669,   586,
          328,  5994,    47,   214,    10,  1498, 37239,    50,   546,     7,
         1477,   110,  8326,  2417,     6,    42,   768,    16,  1887,    13,
           47,     4, 13231,     5,   768,     6,    47,    40,  1532,  4499,
         8326,  2417,   215,    25, 31886,     8, 18434,     4,  1541, 25508,
           40,  4704,    47,   149, 10813,  1377,     8,   694,   588,    12,
          958,     4,  4028,  1294,    40,  1325,    10, 10921,     9,  5687,
           23,     5,   253,     9,     5,     4,    20,   746,   701,    13,
            5,  9759, 21669,    16,    68,  6115,     6,    61,  1171, 12263,
            6,  2799,     6,     8,    70,  2239,  3183,     4,     2],
       device='cuda:0')


In [44]:
no_noise_text_output = tokenizer.decode(no_noise_outputs[0], skip_special_tokens=True)
print(no_noise_text_output)

Welcome to our online programming bootcamp program! Whether you're a complete beginner or looking to improve your programming skills, this course is designed for you. Throughout the course, you will learn essential programming skills such as Python and JavaScript. Our instructors will guide you through interactive projects and provide real-time. Each student will receive a certificate of completion at the end of the. The total cost for the bootcamp is $150, which includes tuition, books, and all learning materials.


In [45]:
print('Original Text:')
display(HTML(f"<p style='font-size:15px; font-family:\"Comic Sans MS\", cursive;'> {original_text}</p>"))
print('\n')

print('Without Noise:')
display(HTML(f"<p style='font-size:15px; font-family:\"Comic Sans MS\", cursive;'>{no_noise_text_output}</p>"))
print('\n')

Original Text:




Without Noise:


**Note: Masked tokens which are at end of the sentence are not being filled in whereas mid of sentence tokens are**